In [0]:
import tensorflow as tf
import numpy as np

In [0]:
from sklearn.datasets import load_iris

In [121]:
x = load_iris().data
y = load_iris().target

x[:10], y[:10]
np.unique(y)

array([0, 1, 2])

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_text, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [0]:
from keras.utils import to_categorical
y = to_categorical(y)

In [0]:
with tf.name_scope('input'):
  input_features = tf.placeholder(dtype=tf.float32, shape=[None, x.shape[1]], name='input_features')
  input_labels = tf.placeholder(dtype=tf.float32, shape=[None, 3], name='input_labels')
  
with tf.name_scope('model'):
  weights = tf.Variable(tf.random_normal(shape=[x.shape[1], 3]), name='weights')
  biases = tf.Variable(tf.random_normal(shape=[3]))
  linear_model = tf.add(tf.matmul(input_features, weights), biases)
  predictions = tf.nn.softmax(linear_model)

with tf.name_scope('training_ops'):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=linear_model, labels=input_labels))
  train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
  tf.summary.scalar(name='loss', tensor=loss)
  
with tf.name_scope('metrics'):
  correct_prediction = tf.cast(tf.equal(tf.argmax(predictions, 1),
                                       tf.argmax(input_labels, 1)), tf.float32)
  accuracy_op = tf.reduce_mean(correct_prediction)
  tf.summary.scalar(name='accuracy', tensor=accuracy_op)

  
  

In [0]:
def next_batch(batch_size, features, labels):
  indices = np.arange(start=0, stop=features.shape[0])
  np.random.shuffle(indices)
  indices = indices[:batch_size]
  return features[indices], labels[indices]

In [126]:
!rm -rf logs/
!rm ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-08-17 04:23:24--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.73.140.127, 52.72.251.164, 54.156.237.249, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.73.140.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  --.-KB/s    in 0.1s    

2018-08-17 04:23:25 (43.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {0} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [129]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://e8445460.ngrok.io


In [130]:
init_op = tf.global_variables_initializer()
batch_size = 15

with tf.Session() as sess:
  
  sess.run(init_op)
  summary = tf.summary.merge_all()
  writer = tf.summary.FileWriter(logdir='./logs', graph=tf.get_default_graph())

  for epoch in range(10):
    for index in range(int(x.shape[0]/batch_size)):
      mini_batch_x, mini_batch_y= next_batch(batch_size=batch_size, features=x, labels=y)

      _, train_loss = sess.run([train_op, loss], feed_dict={input_features: mini_batch_x, input_labels: mini_batch_y})

    print('Epoch: {0}, loss : {1}'.format(epoch, train_loss))

Epoch: 0, loss : 5.248170852661133
Epoch: 1, loss : 7.680081367492676
Epoch: 2, loss : 4.932286739349365
Epoch: 3, loss : 4.876423358917236
Epoch: 4, loss : 7.156226634979248
Epoch: 5, loss : 3.299905300140381
Epoch: 6, loss : 4.476098537445068
Epoch: 7, loss : 7.7392048835754395
Epoch: 8, loss : 5.140228271484375
Epoch: 9, loss : 5.757898807525635
